# Ta feng Data

In [1]:
%load_ext autoreload
%autoreload 2

import os
import random
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
# for reproducibility
def seed_everything(seed=1234):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything()

In [3]:
filename = "transaction.dat"
root_dir = "./data/ta_feng/"
csv_file = "ta_feng_all_months_merged.csv"

df = pd.read_csv(root_dir+csv_file)

In [4]:
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
1,11/1/2000,418683,45-49,115,120107,4710857472535,1,48,46
2,11/1/2000,1057331,35-39,115,100407,4710043654103,2,142,166
3,11/1/2000,1849332,45-49,Others,120108,4710126092129,1,32,38
4,11/1/2000,1981995,50-54,115,100205,4710176021445,1,14,18
...,...,...,...,...,...,...,...,...,...
817736,2/28/2001,312790,35-39,114,530501,4713317035042,2,80,118
817737,2/28/2001,57486,40-44,115,530209,4710731060124,1,40,55
817738,2/28/2001,733526,>65,Unknown,510539,4716340052307,1,78,115
817739,2/28/2001,173704,45-49,115,520457,4714276145315,1,90,96


In [5]:
customer_id = set(df["CUSTOMER_ID"])

## Change Data structure
Data structure is \
[(customer id, transaction),...]

In [6]:
%time
data = []
trans = []  # for removing stop word items
n_trans = 0 # before removing fewer trans and item 
cnt = 0

for i in customer_id:
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    n_trans += len(trans_date)
    # print("custormer ID    :", i, "\n" \
    #       "transaction date:", trans_date, len(trans_date))
    
    if len(trans_date) < 4:
        cnt += 1
        continue

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        
        # Remove users whose transactions are fewer than 3.
        if len(set(product["PRODUCT_ID"])) > 2:
            trans.append(list(product["PRODUCT_ID"]))
            # data.append((i, set(product["PRODUCT_ID"])))
            data.append([i, set(product["PRODUCT_ID"])])
        else:
            cnt += 1

CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 3.1 µs


## Transaction from 0 to 3

In [7]:
data[0:3]

[[1835021,
  {4710094004711,
   4710105002019,
   4710105031613,
   4710114361114,
   4710421090059,
   4710543282318,
   4711390436916,
   4713045519333,
   4715140032311,
   4716447022203}],
 [1835021,
  {4710015102946,
   4710431324236,
   4710431330282,
   4710431397056,
   4713691062900,
   4714005052013,
   4715190002999,
   4715833010893,
   4715833014310}],
 [1835021,
  {3228021990293,
   4710088434593,
   4710105031613,
   4710114362029,
   4710121023302,
   4714220680091,
   4714220680107}]]

## # transactions

In [8]:
len(data)

59191

## # items

In [9]:
itemset = set()
for i in data:
    itemset |= i[1]
    
print(len(itemset))
n_item = len(itemset)

21886


## # removed transactions

In [10]:
cnt

46373

## Average # item in a transaction

In [11]:
mean = 0
for i in data:
    mean += len(i[1])
    
mean /= len(data)
mean

8.334865097734452

## Remove stop words

### Counting items

In [12]:
item_dct = {item : 0 for item in itemset}
for t in trans:
    for item in t:
        item_dct[item] += 1

### Threshold

In [13]:
th = int(0.05*len(data))
th

2959

In [14]:
removed_items = {item for item, cnt in item_dct.items() if cnt>th}
print("The number of removed items:", len(removed_items), \
      "\nRemoved items \n", removed_items)

The number of removed items: 2 
Removed items 
 {4714981010038, 4711271000014}


### Remove

#### Remove from itemset

In [15]:
print("Before remove:", len(itemset))
itemset -= removed_items
print("After remove :", len(itemset))

Before remove: 21886
After remove : 21884


#### Remove from transaction

In [16]:
for i in data:
    i[1] -= removed_items

#### Remove fewer item transaction

In [24]:
data = [(i[0], i[1]) for i in data if len(i[1]) > 2]
print("# Data :", len(data))

itemset = set()
for i in data:
    itemset |= i[1]
print("# Items:", len(itemset))

# Data : 58237
# Items: 21878


#### Convert list to tuple

In [26]:
data = [(i[0], i[1]) for i in data]
print(len(data))

58237


#### Make sure if items were removed

In [27]:
tmp = set()
for i in data:
    tmp |= i[1]

print(len(tmp), len(itemset))
assert len(tmp)==len(itemset)
del tmp

21878 21878


## Save dataset 

In [28]:
filehandler = open(root_dir+filename, "wb") 
pickle.dump(data, filehandler)

## Load dataset from pickle

In [106]:
import pickle
filename = "transaction.dat"
root_dir = "./data/ta_feng/"

f = open(root_dir+filename,"rb")
data = pickle.load(f)

In [107]:
trans = []
for i in data:
    trans.append(list(i[1]))
len(trans)

58237

## Make triple dataset for <br> BASKET-SENSITIVE FACTORIZATION MACHINE(BFM)

### Split dataset for train, test and validation
Ratio is \
train : test : valid = 6 : 2 : 2

# <span style="color: red; ">TODO:</span> split based on user

In [92]:
# dataset ratio
r_train = 0.6
r_test = 0.2
r_valid = 0.2

最初にuserをキーでtransactionのlistがvalueの辞書型を作ってsplitしたほうがいい?\
とりあえずやってみる

In [96]:
udata = {usr : [] for usr in usr_dct.keys()}

for d in data:
    udata[d[0]].append(d[1])

In [104]:
cnt = 0
for u, t in udata.items():
    if len(t) < 3: 
        cnt+=1
print(cnt)

1319


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
seed=1234

# Shuffle data
data = shuffle(data, random_state=seed)

# Split train, test and validation
train, test = train_test_split(data, test_size=0.4, random_state=seed)
test, valid = train_test_split(test, test_size=0.5, random_state=seed)

In [32]:
len(train), len(test), len(valid)

(34942, 11647, 11648)

### Calculate pmi
We only need pmi for train dataset, so we calculated train dataset pmi with original dataset(valiable is `data`).

### Make negative sample
Negative sampleはPositive sampleをベースに作られている．\
1つのPositive sampleにつき2つ作る．つまりNegative sampleはPositive sampleの数の2倍になる．\
Negative sampleはベースとなっているアイテム数(|B|+v)と同じになっている．\
Negative sampleのアイテムはそのUserが購入していないアイテムで構成される．

#### Itemset per user(customer)
`usr_dct`はkeyがuserでvalueがそのuserが購入したitemの集合が入っている． \
だから，`itemset - usr_dct[usr]`でusrが購入していないアイテム集合を取得できる．

In [33]:
import copy

usr_dct = {}
tmp = 0
for i in data:
    if not i[0] in usr_dct.keys():
         # idk why but if I don't use deepcopy, set in data is changed
        usr_dct[i[0]] = copy.deepcopy(i[1])
    else:
        usr_dct[i[0]] |= i[1]

n_usr = len(usr_dct.keys())
print(n_usr)

9956


#### Make negative sample

In [72]:
import tqdm

neg_train = []
for i in tqdm.tqdm(train):
    # target user
    usr = i[0]
    
    # item length
    l_item = len(i[1])
    
    # negative itemset
    neg_itemset = itemset - usr_dct[usr]
    neg_data = random.sample(neg_itemset, k=l_item*2)
    neg_train.append((usr, neg_data[:l_item]))
    neg_train.append((usr, neg_data[l_item:]))

100%|██████████| 34942/34942 [00:25<00:00, 1355.38it/s]


In [84]:
assert 2*len(train)==len(neg_train), \
       f"""The number of negative sample should be 2*train.
                But got train sample:{len(train)}, negative sample:{len(neg_train)}"""

##### Save negative sample dataset

In [86]:
filehandler = open(root_dir+"negative_sample.dat", "wb") 
pickle.dump(neg_train, filehandler)

---
---
---
---

#### Adjacency matrix

In [27]:
import tqdm

def make_adj(num_items, itemset, transactions):
    adjMatrix = np.zeros((num_items, num_items), dtype=np.float32)
    for t in tqdm.tqdm(transactions):
        for i, item1 in enumerate(t):
            # item1がtrain_itemsetに入ってなかったら飛ばす
            if not item1 in itemset: break
                
            sub_t = t[i+1:] # こっちはただの隣接行列
            idx1 = itemset.index(item1)
            for item2 in sub_t:
                # item2がtrain_itemsetに入ってなかったら飛ばす
                if not item2 in itemset: break
                idx2 = itemset.index(item2)
                adjMatrix[idx1][idx2] += 1

    return adjMatrix.astype(np.int32)

##### Train itemset

In [28]:
train_itemset = set()
for i in train:
    train_itemset |= i[1]

train_itemset = list(train_itemset)
print(len(train_itemset))
n_train_item = len(train_itemset)

19865


##### Whole transaction is `trans`

In [29]:
adj = make_adj(n_train_item, train_itemset, trans)

100%|██████████| 59191/59191 [19:38<00:00, 50.24it/s]  


##### Save adjacency matrix as numpy object

In [31]:
print(adj)
np.save(f"{root_dir}adj", adj)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [116]:
train[0]

(293044,
 {20570125,
  4710043100082,
  4710731070192,
  4712172200091,
  4714008271008,
  4719864067116,
  5998710132430,
  5998710132461})

In [105]:
# 内包表記のほうが速いっぽい
# ref: https://utgwkk.hateblo.jp/entry/2017/03/09/154314

# custormer id == 1835021
%time
custormer_id = 1835021
# target_custormer = lambda x: x[0]==custormer_id
# u_data = map(lambda x: x[1] if x[0]==custormer_id else None, trans_from_pickle)
# u_data = list(filter(target_custormer, trans_from_pickle))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [106]:
%time
target_tran = [i[1] for i in data if i[0]==custormer_id]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [107]:
target_tran

[{4710094004711,
  4710105002019,
  4710105031613,
  4710114361114,
  4710421090059,
  4710543282318,
  4711390436916,
  4713045519333,
  4715140032311,
  4716447022203},
 {3228021990293,
  4710088434593,
  4710105031613,
  4710114362029,
  4710121023302,
  4714220680091,
  4714220680107},
 {20554705,
  29000070295,
  4710105031613,
  4710205005750,
  4710731060124,
  4710740600090,
  4710740600106,
  8851954102126},
 {20412074, 4710063312168, 4710088433268},
 {4710015102946,
  4710431324236,
  4710431330282,
  4710431397056,
  4713691062900,
  4714005052013,
  4715190002999,
  4715833010893,
  4715833014310}]